# Prep

In [1]:
import musicbrainzngs
import pandas as pd

from ratelimit import limits, sleep_and_retry
import traceback

import numpy as np

# Authentication

In [2]:
musicbrainzngs.set_useragent(app="Opus Metadata Fetcher",version = "1.0")

# Add Musicbrainz metadata

## Utility Functions

In [32]:
def mb_recording_template(input_isrc = ''):
    # default recording_id in the format of music brainz recording id
    default_recording_id = 'zzzzzzzz-zzzz-zzzz-zzzz-zzzzzzzzzzzz'
    default_mb_recording_title = ''
    default_mb_recording_length = '0'
    
    data = {
    'mb_recording_id': [default_recording_id],
    'mb_recording_title': [default_mb_recording_title],
    'mb_recording_length': [default_mb_recording_length],
    'isrc_opus': [input_isrc]
}
    default_recording_df = pd.DataFrame(data)
    
    return default_recording_df

def compare_dataframes(df1, df2):
    # Check if column names are equal
    if list(df1.columns) != list(df2.columns):
        print("Column names are not equal")
        return False
    
    # Check if column types are equal
    if not df1.dtypes.equals(df2.dtypes):
        print("Column types are not equal")
        return False
    
    # print("Column names and types are equal")
    return True
def create_mb_globals():
    # dictionary of globals
    
    # default df
    mb_recording_df_dflt = mb_recording_template()
    
    found_recording_dfs = []
    missing_recording_dfs = []
    mb_release_dfs = []
    mb_label_dfs = []
    related_recording_dfs = []
    related_isrc_dfs = []
    error_log = []
    

    mb_global_dict = {'recording_df_default':mb_recording_df_dflt,
                      'found_recording_dfs':found_recording_dfs,
                      'missing_recording_dfs':missing_recording_dfs,
                      'release_dfs':mb_release_dfs,
                      'label_dfs':mb_label_dfs,
                      'related_recording_dfs':related_recording_dfs,
                      'related_isrc_dfs':related_isrc_dfs,
                      'error_log':error_log}
    return mb_global_dict




def format_recording(mb_recording,input_isrc):
    # formats music brains recording object
    recording_df = pd.DataFrame(mb_recording['isrc']['recording-list'])
    recording_df['isrc_opus'] = input_isrc
    recording_df = recording_df.rename(columns={'id':'mb_recording_id',
                                                    'title':'mb_recording_title',
                                                    'length':'mb_recording_length'})
    recording_df = recording_df[['mb_recording_id','mb_recording_title','mb_recording_length','isrc_opus']]
    return recording_df

In [4]:
# consolidate dataframes


### Release

In [5]:
def parse_release_text(text):
    """retrieve language from text-representation field of release dataframe
    ex: {'language': 'eng', 'script': 'Latn'} --> 'eng' """
    try: 
        # format string as a dictionary and extract value of 'language' key
        if pd.isna(text):
            language = 'zzz'
        else:
            language = dict(text)['language']
    except TypeError as e:
        language = 'zzz'
        print(e)
    finally:
        return language
def parse_release_date(date_str):
    # handle missing values
    if pd.isna(date_str):
        return '1900-01-01'
    elif len(date_str) == 4:
        return date_str + '-01-01'  # Assuming it's the start of the year
    elif len(date_str) == 7:
        return date_str + '-01'
    else:
        return date_str
    
def flag_complete_date(date_str):
    try:
        pd.to_datetime(date_str,format='%Y-%m-%d')
        return '1'
    except ValueError:
        return '0'
    except TypeError:
        return '0'

def format_mb_release(mb_release,recording_id,isrc):
    release_df = pd.DataFrame(mb_release['release-list'])
    # print(mb_release.columns)
    # get release language from text-representation field
    release_df['mb_release_language'] = release_df['text-representation'].apply(parse_release_text)
    # flag date format
    release_df['mb_complete_date_flag'] = release_df['date'].apply(flag_complete_date)
    # format date
    release_df['mb_date'] = release_df['date'].apply(parse_release_date)
    
    # add recording_id
    release_df['mb_recording_id'] = recording_id
    
    # persist isrc
    release_df['isrc_opus'] = isrc
    # rename columns
    release_df = release_df.rename(columns = {'id':'mb_release_id',
                                              'title':'mb_release_title',
                                              'status':'mb_release_status',
                                              'country':'mb_release_country',
                                              'quality':'mb_release_quality'})
    
    release_df['recording_id'] = recording_id
    # select columns
    release_df = release_df[['mb_release_id',
                             'mb_release_title',
                             'mb_release_status',
                             'mb_release_country',
                             'mb_release_quality',
                             'mb_release_language',
                             'mb_complete_date_flag',
                             'mb_date',
                             'mb_recording_id',
                             'isrc_opus']]
    return release_df
    # print('done')


In [6]:
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
# define function
def build_release_df(recording_record,mb_global_dict):
    # print(recording_record)
    recording_id = recording_record['mb_recording_id']
    isrc = recording_record['isrc_opus']
    # if HTTPError, handle with default df
    # try:
        # look up release using recording_id
    print('searching for release')
    releases = musicbrainzngs.browse_releases(recording=recording_id)
    print('formatting release')
    release_df = format_mb_release(releases,recording_id=recording_id,isrc = isrc)
    print('adding to global dict')
    mb_global_dict['release_dfs'].append(release_df)
    # except musicbrainzngs.musicbrainz.ResponseError as e:
    #     print('recording not found in musicbrainz')

    # finally:
    #     return mb_global_dict

## Build recording df

In [7]:
# plan
mb_global_dict = create_mb_globals()
# isrc --> recording --> release --> label
input_isrcs = ['USSM10600677',
               'USSM106006771',
               'USSM106006772',
               'USSM106006773',
               'USSM106006774',
               'USSM106006775']
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
def build_recording_df(input_isrc,mb_global_dict):
    print(input_isrc)
    # if HTTPError, handle with default df
    try:
        # look up recording using isrc. if the isrc is not found, ResponseError will be thrown
        recordings = musicbrainzngs.get_recordings_by_isrc(input_isrc, includes=[], release_status=[], release_type=[])
        # if isrc is found, then format the recording information
        recording_df = format_recording(recordings,input_isrc = input_isrc)
        assert compare_dataframes(recording_df,mb_global_dict['recording_df_default'])
        # add to list of found recordings
        mb_global_dict['found_recording_dfs'].append(recording_df)
    except musicbrainzngs.musicbrainz.ResponseError as e:
        print('isrc not found in musicbrainz')
        # create missing recording info for isrc
        recording_df = mb_recording_template(input_isrc=input_isrc)
        # ensure proper format 
        assert compare_dataframes(recording_df,mb_global_dict['recording_df_default'])
        # add to missing recording dfs
        mb_global_dict['missing_recording_dfs'].append(recording_df)
    
    # except Exception as e:
    #     print(e)
    #     error_log = {'process':'recording_df',
    #                  'error_type':'fatal',
    #                  'error':e}
    #     mb_global_dict['error_log'].append(error_log)
        # raise e
        
    
    finally:
        return mb_global_dict

# dim_song_detail --> stg_mb_recording_detail (key: isrc, mb_recording_id)
# stg_mb_release_detail (key: recording/release)
# stg_mb_label_detail (key: label/release)

In [33]:
# test one isrc
# input_isrc = 'US23S9528112'
# recordings = musicbrainzngs.get_recordings_by_isrc(input_isrc)
# recording_df = format_recording(recordings,input_isrc = input_isrc)
# recording_df

,mb_recording_id,mb_recording_title,mb_recording_length,isrc_opus
0,704ac8b9-5ee6-4c36-ab5d-5a72332b52e7,1st of tha Month,314466,US23S9528112


## Label Functions

In [8]:
def format_mb_label(label_info,release_record):
    # add release/recording id
    mb_release_id = release_record['mb_release_id']
    mb_recording_id = release_record['mb_recording_id']
    isrc = release_record['isrc_opus']
    
    # format label from API call
    label_df = pd.DataFrame(label_info['label-list'])
    
    # add release/recording info
    label_df['mb_release_id'] = mb_release_id
    label_df['mb_recording_id'] = mb_recording_id
    label_df['isrc_opus'] = isrc
    
    # rename columns
    label_df = label_df.rename(columns = {'id':'mb_label_id',
                                          'name':'mb_label_name',
                                          'country':'mb_label_country',
                                          })
    
    # select columns
    label_df = label_df[['mb_label_id','mb_release_id','mb_recording_id','mb_label_name','mb_label_country','isrc_opus']]
    
    return(label_df)

In [9]:
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
# define function
def build_label_df(release_record,mb_global_dict):
    # input: row from release_df, mb_global_dict
    # output: record for label_df added to global_dict
    release_id = release_record['mb_release_id']
    # if HTTPError, handle with default df
    try:
        # look up recording using isrc. if the isrc is not found, ResponseError will be thrown
        label_info = musicbrainzngs.browse_labels(release=release_id)
        label_df = format_mb_label(label_info,release_record)
        mb_global_dict['label_dfs'].append(label_df)
        # mb_global_dict['release_dfs'].append(release_df)
    except musicbrainzngs.musicbrainz.ResponseError as e:
        print('label not found in musicbrainz')

    finally:
        return mb_global_dict

## Related Songs

In [10]:
def format_related_recording_df(recording):
    # create dataframe
    related_recording_df = pd.DataFrame(recording['recording']['recording-relation-list'])
    # rename
    related_recording_df = related_recording_df.rename(columns={'type':'mb_relationship_type',
                                                                'type-id':'mb_relationship_type_id',
                                                                'target':'mb_recording_id_target',
                                                                'direction':'mb_relation_direction'})
    related_recording_df['mb_recording_id_source'] = recording['recording']['id']
    
    # add source isrc
    isrc_dfs = []
    if 'isrc-list' in recording['recording']:
        # add isrc:
        for isrc in recording['recording']['isrc-list']:
            # Copy the dataframe
            df_copy = related_recording_df.copy()
            # Add a new column containing the item from the list
            df_copy['mb_source_isrc'] = isrc
            # Store the modified dataframe in the list
            isrc_dfs.append(df_copy)
        
        isrc_df = pd.concat(isrc_dfs)
    
    else:
        isrc_df = related_recording_df
        isrc_df['mb_source_isrc'] = 'ZZZZZZZZZZZZ'
        
    # select columns
        
    
    return isrc_df

In [11]:
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
def build_related_df(recording_record,mb_global_dict):
    # if HTTPError, handle with default df
    recording_id = recording_record['mb_recording_id']
    # look up recording using isrc. if the isrc is not found, ResponseError will be thrown
    recording = musicbrainzngs.get_recording_by_id(recording_id, includes=['recording-rels','isrcs'], release_status=[], release_type=[])
    # if related recording list exists, parse
    if 'recording-relation-list' in recording['recording']:
        print('parse')
        # related_recording_info = recording['recording']['recording-relation-list']
        # print(related_recording_info[0].keys())
        
        recording_df = format_related_recording_df(recording)
        
        mb_global_dict['related_recording_dfs'].append(recording_df)
        return mb_global_dict
        # related_recording_df = format_related_recording_df(related_recordings)
    else:
        print('no related recordings')
        return mb_global_dict


In [12]:
def add_target_isrcs(df,isrc_list):
    target_dfs = []
    for isrc in isrc_list:
        isrc_df = df.copy()
        print(isrc)
        isrc_df['target_isrc'] = isrc
        target_dfs.append(isrc_df)
    
    target_df = pd.concat(target_dfs) 
    return target_df

In [13]:
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
def build_related_isrc_df(related_recording_record,mb_global_dict):
    recording_id = related_recording_record['mb_recording_id_target']
    # print(recording_id)
    target_recording = musicbrainzngs.get_recording_by_id(id=recording_id,includes=['isrcs'])
    print(target_recording)
    if 'isrc-list' in target_recording['recording']:
        print("at least one isrc found")
        # target_df = add_target_isrcs(related_df,target_recording['recording']['isrc-list'])
        if int(target_recording['recording']['isrc-count']) >= 1:
            # print('more than one isrc found')
            isrc_records = []
            for isrc in target_recording['recording']['isrc-list']:
                # create one copy of record with each isrc as target isrc
                recording_copy = related_recording_record.copy()
                recording_copy['target_isrc'] = isrc
                isrc_records.append(recording_copy)
                print(isrc)
            isrc_df = pd.DataFrame(isrc_records)
            mb_global_dict['related_isrc_dfs'].append(isrc_df)
            return mb_global_dict
    else:
        print("no isrc found")
        isrc_records = []
        recording_copy = related_recording_record.copy()
        
        target_isrc = 'ZZZZZZZZZZZZ'
        recording_copy['target_isrc'] = target_isrc
        isrc_records.append(recording_copy)
        isrc_df = pd.DataFrame(isrc_records)
        mb_global_dict['related_isrc_dfs'].append(isrc_df)
        return mb_global_dict
    pass

In [14]:
# Call the function ensuring one call per second
# for isrc in input_isrcs:
#     mb_global_dict = build_recording_df(input_isrc=isrc,mb_global_dict = mb_global_dict)


## Build release df

In [15]:
# for df in mb_global_dict['found_recording_dfs']:
#     for recording_id in df['mb_recording_id']:
#         build_release_df(recording_id,mb_global_dict)

In [16]:
# mb_global_dict['release_dfs'][0]

# Scratch Work

In [17]:
# test = mb_global_dict['release_dfs'][0]
# test

In [18]:
# label_info = musicbrainzngs.browse_labels(release='27af023b-7967-4cc4-8ef5-7edccada8219')

In [19]:


# format_mb_label(label_info, test.iloc[0])

In [20]:
# release_record = mb_global_dict['release_dfs'][0].iloc[0]
# release_record

# build_label_df(release_record,mb_global_dict)

In [21]:
# mb_global_dict.keys()

## end to end

In [34]:
# plan
# initialize global dictionary to store dataframes
mb_global_dict = create_mb_globals()
# isrc --> recording --> release --> label
input_isrcs = ['US23S9528103',
'USPO19000001',
'US23S9528112',
'USPO10000877',
'USPO18700001',
'USIR19915096',
'USRE49800185',
'USPO19100007',
'US23S9333103',
'USPO19100005',
'USIR19915088',
'USIR19915072',
'USIR19915082',
'US23S9728128',
'US23S9333107',
'USPO18700101',
'USPO10000853',
'US23S9529113',
'US23S9528107',
'USSM10101440',
'US23S0028112',
'USPO19900508',
'US23S0846109',
'US23S9833111',
'US23S9428106',
'US23S9528104',
'US23S9528101',
'US23S9528114',
'USPO18700108',
'USPO10000882',
'US23S9528105',
'US23S9333108',
'US23S9428105',
'US23S9528102',
'USPO10000854',
'USPO10000086',
'US23S0028105',
'USPO18800002',
'USPO19600015',
'USPO10000891',
'USPO10000879',
'US23S9428107',
'USIR19915077',
'USPO10000873',
'USPO10000874',
'USUYG1070995',
'US23S9333105',
'US23S9728102',
'US23S9728115',
'US23S0228104',
'USIR19915087',
'USPO10000881',
'USPO19100015',
'US23S9728111',
'USPO10000875',
'US23S9728104',
'US23S9728108',
'US23S9333106',
'US23S9833104',
'US23S9528110',
'USIR19915079',
'US23S9729103',
'US23S9528109',
'US23S9528116',
'US23S9528106',
'USPO10000858',
'US23S9528111',
'US23S9428104',
'USPO10000851',
'USBB40706439',
'US23S9833102',
'US23S0846106',
'USPO10000883',
'US23S0833112',
'US23S9333102',
'US23S0028109',
'US23S9528117',
'US23S9833105',
'US23S9833103',
'US23S9333104',
'USIR19905031',
'USPO10000850',
'USSM10101450',
'USPO10000872',
'USPO18800004',
'USPO19600010',
'US23S9528108',
'US23S9833107',
'USIR10001351',
'USIR19915078',
'USIR19915071',
'US23S9728114',
'US23S9428103',
'USPO10000855',
'US23S0028111',
'USSM10016341',
'USIR19915074',
'US23S9833112',
'USIR19915073']
# 
# create recording dfs
for isrc in input_isrcs:
    mb_global_dict = build_recording_df(input_isrc=isrc,mb_global_dict = mb_global_dict)

mb_global_dict['recording_df'] = pd.concat(mb_global_dict['found_recording_dfs'])

mb_global_dict['recording_df']

# build related songs from recorded df


US23S9528103
USPO19000001
US23S9528112
USPO10000877
USPO18700001
USIR19915096
USRE49800185
isrc not found in musicbrainz
USPO19100007
US23S9333103
isrc not found in musicbrainz
USPO19100005
USIR19915088
USIR19915072
USIR19915082
US23S9728128
isrc not found in musicbrainz
US23S9333107
USPO18700101
USPO10000853
US23S9529113
isrc not found in musicbrainz
US23S9528107
USSM10101440
isrc not found in musicbrainz
US23S0028112
isrc not found in musicbrainz
USPO19900508
US23S0846109
US23S9833111
US23S9428106
isrc not found in musicbrainz
US23S9528104
US23S9528101
US23S9528114
USPO18700108
USPO10000882
US23S9528105
US23S9333108
isrc not found in musicbrainz
US23S9428105
isrc not found in musicbrainz
US23S9528102
USPO10000854
USPO10000086
US23S0028105
isrc not found in musicbrainz
USPO18800002
USPO19600015
USPO10000891
USPO10000879
US23S9428107
USIR19915077
USPO10000873
USPO10000874
USUYG1070995
isrc not found in musicbrainz
US23S9333105
isrc not found in musicbrainz
US23S9728102
isrc not found i

,mb_recording_id,mb_recording_title,mb_recording_length,isrc_opus
0,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Eternal,243893,US23S9528103
0,828debce-b03a-4560-bb71-8454494c3c41,100 Miles and Runnin’,272933,USPO19000001
0,704ac8b9-5ee6-4c36-ab5d-5a72332b52e7,1st of tha Month,314466,US23S9528112
0,94d04653-dd81-4abb-88d2-46de576147f3,8 Ball (remix),214000,USPO10000877
1,c9dd80d2-7473-497d-8499-8ed609f98d25,8 Ball (remix),292906,USPO10000877
...,...,...,...,...
0,a41cae1f-1417-4f48-8795-cf3343b04fde,We Want Eazy,301000,USPO10000855
0,f09d0b4c-27e9-4cc8-be33-d469bd64c83e,Weedman,237093,USSM10016341
0,70b858f3-da52-4d26-8dd8-0d5aa55dfe66,What’s the Difference,244240,USIR19915074
0,c3cff5fe-7427-4fb5-b7f3-58f843928939,Wut Would You Do,351640,US23S9833112


In [35]:
for idx, recording_record in mb_global_dict['recording_df'].iterrows():
    # print(recording_id)
    # print(recording_record)
    build_release_df(recording_record,mb_global_dict)

searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to global dict
searching for release
formatting release
adding to glob

In [24]:
# recording_record = mb_global_dict['recording_df'].iloc[0]
# recording_id = recording_record['mb_recording_id']
# isrc = recording_record['isrc_opus']
# # if HTTPError, handle with default df
# # try:
#     # look up release using recording_id
# print('searching for release')
# releases = musicbrainzngs.browse_releases(recording=recording_id)
# print('formatting release')
# release_df = format_mb_release(releases,recording_id=recording_id,isrc=isrc)
# print('adding to global dict')
# mb_global_dict['release_dfs'].append(release_df)

In [36]:
mb_global_dict['release_df'] = pd.concat(mb_global_dict['release_dfs'])

In [37]:
for idx,row in mb_global_dict['release_df'].iterrows():
    build_label_df(row,mb_global_dict)

In [38]:
mb_global_dict['label_df'] = pd.concat(mb_global_dict['label_dfs'])

In [39]:
mb_global_dict['label_df'].shape

(1120, 6)

## Samples/Song relationships

In [40]:
# mb_global_dict['recording_df']
# mb_global_dict['release_df']

In [41]:
# test = musicbrainzngs.get_release_by_id('5fe133cb-d394-4089-a5e1-ec1e27bf4fe3', includes=['work-rels'], release_status=[], release_type=[])
# test

In [42]:
# test = musicbrainzngs.browse_releases(recording='2c0b2787-307c-49e9-b2e9-5b1ddd6e713e',includes = ['recording-rels'])

In [43]:
# test['release-list'][15]

In [44]:
# musicbrainzngs.get_release_by_id('4102ba3c-a576-43f5-bc38-49c5b42ef03a',includes=['recording-rels'])

In [45]:
# mb_global_dict['recording_df']

In [46]:

for idx,row in mb_global_dict['recording_df'].iterrows():    
    mb_global_dict = build_related_df(row,mb_global_dict)


no related recordings
parse
parse
no related recordings
parse
no related recordings
parse
no related recordings
no related recordings
no related recordings
no related recordings
parse
parse
no related recordings
no related recordings
parse
parse
no related recordings
no related recordings
no related recordings
no related recordings
no related recordings
no related recordings
parse
no related recordings
no related recordings
no related recordings
parse
no related recordings
no related recordings
parse
parse
parse
parse
no related recordings
no related recordings
no related recordings
no related recordings
parse
no related recordings
parse
parse
parse
no related recordings
no related recordings
parse
no related recordings
parse
no related recordings
parse
no related recordings
no related recordings
parse
parse
no related recordings
parse
no related recordings
no related recordings
no related recordings
parse
no related recordings
no related recordings
parse
no related recordings
no relat

In [47]:


# related_df = format_related_recording_df(recording)
# add target isrc

In [48]:
# 
# if 'isrc-list' in target_recording['recording']:
#     print("at least one isrc found")
#     target_df = add_target_isrcs(related_df,target_recording['recording']['isrc-list'])
#     if int(target_recording['recording']['isrc-count']) >= 1:
#         print('more than one isrc found')
#     elif int(target_recording['recording']['isrc-count']) == 1:
#         print('one isrc found')
# else:
#     print("no isrc found")

In [49]:


# add_target_isrcs(related_df,target_recording['recording']['isrc-list'])

In [50]:
mb_global_dict['related_recording_df'] = pd.concat(mb_global_dict['related_recording_dfs'])

In [51]:


for idx,row in mb_global_dict['related_recording_df'].iterrows():
    build_related_isrc_df(row,mb_global_dict)

{'recording': {'id': '9f07f5f2-9b7b-4e65-8366-c8aa706f5039', 'title': 'Lend Me an Ear', 'length': '200533', 'isrc-list': ['USAT20503494'], 'isrc-count': 1}}
at least one isrc found
USAT20503494
{'recording': {'id': '026c26a7-70b4-4355-83d8-7ca66d5d68b8', 'title': '(You Gotta) Fight for Your Right (to Party)', 'length': '208000', 'isrc-list': ['USDJ20110712', 'USDJ28600007'], 'isrc-count': 2}}
at least one isrc found
USDJ20110712
USDJ28600007
{'recording': {'id': 'bf266c83-4333-495c-a0c0-312c23178721', 'title': 'Theme From Shaft', 'length': '277000', 'isrc-list': ['USFI87100050'], 'isrc-count': 1}}
at least one isrc found
USFI87100050
{'recording': {'id': '8403caf8-3714-46da-934c-1fe91a049540', 'title': 'Thriller', 'length': '357746', 'disambiguation': 'album version', 'isrc-list': ['USSM18200002', 'USSM19902989'], 'isrc-count': 2}}
at least one isrc found
USSM18200002
USSM19902989
{'recording': {'id': '6789cdd9-afe2-4d7d-8cd1-1699e4154b5e', 'title': "3000 Miles & Runnin'", 'length': '2

In [54]:
mb_global_dict['related_isrc_df'] = pd.concat(mb_global_dict['related_isrc_dfs'])


In [55]:
mb_global_dict['related_isrc_df']

,mb_relationship_type,mb_relationship_type_id,mb_recording_id_target,mb_relation_direction,recording,mb_recording_id_source,mb_source_isrc,attribute-list,attributes,ordering-key,target_isrc
0,samples material,9efd9ce9-e702-448b-8e76-641515e8fe62,9f07f5f2-9b7b-4e65-8366-c8aa706f5039,forward,"{'id': '9f07f5f2-9b7b-4e65-8366-c8aa706f5039',...",828debce-b03a-4560-bb71-8454494c3c41,USPO19000001,NaN,NaN,NaN,USAT20503494
1,samples material,9efd9ce9-e702-448b-8e76-641515e8fe62,026c26a7-70b4-4355-83d8-7ca66d5d68b8,forward,"{'id': '026c26a7-70b4-4355-83d8-7ca66d5d68b8',...",828debce-b03a-4560-bb71-8454494c3c41,USPO19000001,NaN,NaN,NaN,USDJ20110712
1,samples material,9efd9ce9-e702-448b-8e76-641515e8fe62,026c26a7-70b4-4355-83d8-7ca66d5d68b8,forward,"{'id': '026c26a7-70b4-4355-83d8-7ca66d5d68b8',...",828debce-b03a-4560-bb71-8454494c3c41,USPO19000001,NaN,NaN,NaN,USDJ28600007
2,samples material,9efd9ce9-e702-448b-8e76-641515e8fe62,bf266c83-4333-495c-a0c0-312c23178721,forward,"{'id': 'bf266c83-4333-495c-a0c0-312c23178721',...",828debce-b03a-4560-bb71-8454494c3c41,USPO19000001,NaN,NaN,NaN,USFI87100050
3,samples material,9efd9ce9-e702-448b-8e76-641515e8fe62,8403caf8-3714-46da-934c-1fe91a049540,forward,"{'id': '8403caf8-3714-46da-934c-1fe91a049540',...",828debce-b03a-4560-bb71-8454494c3c41,USPO19000001,NaN,NaN,NaN,USSM18200002
...,...,...,...,...,...,...,...,...,...,...,...
3,samples material,9efd9ce9-e702-448b-8e76-641515e8fe62,e91d63e0-dc0d-4d0f-800c-ee57e7d573a5,backward,"{'id': 'e91d63e0-dc0d-4d0f-800c-ee57e7d573a5',...",70b858f3-da52-4d26-8dd8-0d5aa55dfe66,USIR19915074,NaN,NaN,NaN,DEA621801857
0,mashes up,579d0b4c-bf77-479d-aa59-a8af1f518958,64112453-b60d-403a-966f-0553b644a179,backward,"{'id': '64112453-b60d-403a-966f-0553b644a179',...",2fcb5bc5-2a28-4b4e-afe5-22a102ee7384,USIR19915073,NaN,NaN,1,ZZZZZZZZZZZZ
1,samples material,9efd9ce9-e702-448b-8e76-641515e8fe62,7535fba6-fca6-4daa-80fe-da60d8cc64d9,backward,"{'id': '7535fba6-fca6-4daa-80fe-da60d8cc64d9',...",2fcb5bc5-2a28-4b4e-afe5-22a102ee7384,USIR19915073,NaN,NaN,NaN,DEUM71906145
2,samples material,9efd9ce9-e702-448b-8e76-641515e8fe62,9b7b3e28-a1ed-4096-ac8f-bcf1e230479d,backward,"{'id': '9b7b3e28-a1ed-4096-ac8f-bcf1e230479d',...",2fcb5bc5-2a28-4b4e-afe5-22a102ee7384,USIR19915073,NaN,NaN,NaN,USDJ20010568
